# 2. CLASSIFICATORE LINEARE SVM CON INPUT DI N-GRAMMI

Classificatore basato su  SVM che prende in input una matrice di features basata su n-grammi di 3 tipi:
1. Caratteri
2. Token
3. Part of Speech

***

### Preprocessing dei dati

Dobbiamo ottenere un vettore di features, basato sulle occorrenze di n-grammi all'interno delle frasi, per ciascun post. Come prima cosa si crea si per training che per test un set formato da liste di dizionari. I dizionari riportano per ogni token 3 campi che corrispondono alla parola, la forma lemmatizzata e la parte del discorso. Infine contiamo all'interno dei post le occorrenze degli n-grammi. Una volta ottenuti tutti gli n-grammi possiamo creare la matrice di occorrenze. 

In [2]:
import pandas as pd
import os

In [3]:
#creiamo un dizionario che abbia 3 campi: word, lemma e part of speech(pos)
def create_set(type):
    annotated_posts = []
    for doc in os.listdir("../data/UD_annotation"):
        if type in doc: 
            doc_tokens = []
            doc_path = "../data/UD_annotation/" + doc
            for line in open(doc_path, "r", encoding="utf-8"):
                splitted = line.strip().split("\t")
                if splitted[0].isdigit() and "-" not in splitted[0]:
                    word=splitted[1]
                    lemma=splitted[2]
                    pos=splitted[3]
                    new_token = {'word': word, 'lemma': lemma, 'pos': pos}
                    doc_tokens.append(new_token)
            annotated_posts.append(doc_tokens)
    return annotated_posts

annotation_tr = create_set("training")
annotation_ts = create_set("test")


In [4]:
print(annotation_tr[0])

[{'word': 'Le', 'lemma': 'il', 'pos': 'DET'}, {'word': '5', 'lemma': '5', 'pos': 'NUM'}, {'word': 'sgradevoli', 'lemma': 'sgradevole', 'pos': 'ADJ'}, {'word': 'realtà', 'lemma': 'realtà', 'pos': 'NOUN'}, {'word': 'di', 'lemma': 'di', 'pos': 'ADP'}, {'word': 'cui', 'lemma': 'cui', 'pos': 'PRON'}, {'word': 'Berlusconi', 'lemma': 'Berlusconi', 'pos': 'PROPN'}, {'word': 'dovrebbe', 'lemma': 'dovere', 'pos': 'AUX'}, {'word': 'render', 'lemma': 'rendere', 'pos': 'VERB'}, {'word': 'si', 'lemma': 'si', 'pos': 'PRON'}, {'word': 'personalmente', 'lemma': 'personalmente', 'pos': 'ADV'}, {'word': 'conto', 'lemma': 'conto', 'pos': 'NOUN'}, {'word': '<URL>', 'lemma': '<URL>', 'pos': 'PROPN'}, {'word': 'Mario', 'lemma': 'Mario', 'pos': 'PROPN'}, {'word': 'Monti', 'lemma': 'Monti', 'pos': 'PROPN'}, {'word': 'non', 'lemma': 'non', 'pos': 'ADV'}, {'word': 'usa', 'lemma': 'usare', 'pos': 'VERB'}, {'word': 'mezzi', 'lemma': 'mezzo', 'pos': 'ADJ'}, {'word': 'termini', 'lemma': 'termine', 'pos': 'NOUN'}]


In [5]:
#contiamo il numero di post(sample) per training e test
print(f'Training: {len(annotation_tr)}')
print(f'Test: {len(annotation_ts)}')

Training: 3977
Test: 872


In [6]:
#funzione che conta le occorrenze degli ngrammi all'interno di un post
def count_ngrams(post, ft_type, n, char=False):
    all_ngrams = {}
    allwords = []
    if not char:
        for word in post:
            allwords.append(word[ft_type])
        post_length = len(allwords)
    else:
        for word in post:
            allwords.append(word['word'])
        allwords = " ".join(allwords)
        post_length = len(allwords)-1
    for i in range(len(allwords)-(n-1)):
        new_ngram = allwords[i:i+n]
        if not char:
            ngram_id = f'{ft_type}_{n}_'+'_'.join(new_ngram)
        else:
            ngram_id = f'char_{n}_'+'_'.join(new_ngram)
        if ngram_id not in all_ngrams:
            all_ngrams[ngram_id] = 1
        else:
            all_ngrams[ngram_id]+=1
    #normalizzare in base alla post_length
    for ngram in all_ngrams:
        all_ngrams[ngram] = all_ngrams[ngram]/post_length
    return all_ngrams

print(count_ngrams(annotation_tr[2], "word", 2, False))



{'word_2_#la7_ma': 0.05555555555555555, 'word_2_ma_perche': 0.05555555555555555, "word_2_perche_'": 0.05555555555555555, "word_2_'_Mario": 0.05555555555555555, 'word_2_Mario_Monti': 0.05555555555555555, 'word_2_Monti_non': 0.05555555555555555, 'word_2_non_fa': 0.05555555555555555, 'word_2_fa_il': 0.05555555555555555, 'word_2_il_premier': 0.05555555555555555, 'word_2_premier_?': 0.05555555555555555, 'word_2_?_Che': 0.05555555555555555, 'word_2_Che_persona': 0.05555555555555555, 'word_2_persona_competente': 0.05555555555555555, 'word_2_competente_e': 0.05555555555555555, 'word_2_e_per': 0.05555555555555555, 'word_2_per_bene': 0.05555555555555555, 'word_2_bene_!': 0.05555555555555555}


Uniamo per ogni tipo di informazione che vogliamo i relativi dizionari, così da avere un dizionario denso di informazioni. 


In [7]:
def get_features(dataset, info_types):
    all_features = []
    for post in dataset: 
        complete_dict = dict()
        for info in info_types:
            info_dict = count_ngrams(post, info[0], info[1], info[2])
            complete_dict = complete_dict | info_dict
        all_features.append(complete_dict)
    return all_features

infos = [['word', 1, False], ['word', 2, False], ['word', 1, True], ['word', 2, True]]

tr_features = get_features(annotation_tr, infos)

print(tr_features[1])
    
        

{'word_1_Mario': 0.043478260869565216, 'word_1_Monti': 0.043478260869565216, 'word_1_:': 0.043478260869565216, "word_1_c'": 0.043478260869565216, 'word_1_è': 0.043478260869565216, 'word_1_il': 0.043478260869565216, 'word_1_rischio...': 0.043478260869565216, 'word_1_di': 0.08695652173913043, 'word_1_trasformare': 0.043478260869565216, "word_1_l'": 0.08695652173913043, 'word_1_Italia': 0.043478260869565216, 'word_1_da': 0.043478260869565216, 'word_1_Stato': 0.08695652173913043, 'word_1_fondatore': 0.043478260869565216, 'word_1_in': 0.043478260869565216, 'word_1_affondatore': 0.043478260869565216, 'word_1_Unione': 0.043478260869565216, 'word_1_europea': 0.043478260869565216, 'word_1_!': 0.043478260869565216, 'word_1_<URL>': 0.043478260869565216, 'word_2_Mario_Monti': 0.043478260869565216, 'word_2_Monti_:': 0.043478260869565216, "word_2_:_c'": 0.043478260869565216, "word_2_c'_è": 0.043478260869565216, 'word_2_è_il': 0.043478260869565216, 'word_2_il_rischio...': 0.043478260869565216, 'word_

Per ridurre la dimensionalità della nostra matrice, eliminiamo gli n-grammi che occorrono sotto a un numero soglia di documenti.

In [8]:
def get_feature_list(feature_set):
    feature_list = []
    for post in feature_set:
        for feature in post:
            if feature not in feature_list:
                feature_list.append(feature)
    return feature_list

64468


In [15]:
all_features = get_feature_list(tr_features)

In [24]:
def is_common(feature, feature_set, min_occurrences):
    count = 0
    for post in feature_set:
        if feature in post:
            count += 1
    if count < min_occurrences:
        return False
    else:
        return True
    
filtered_features = list(filter(lambda x: is_common(x, tr_features, 3), all_features))

In [ ]:
def keep_feature(feature, features_tokeep):
    if feature in features_tokeep:
        return True
    else:
        return False

def filter_dataset(dataset):
    ##filtrare con pop() 
    return



In [25]:
print(len(filtered_features))

7534
